In [130]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
import datetime

In [131]:
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Mobile Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,id;q=0.8,ms;q=0.7,ja;q=0.6,ko;q=0.5"
    }
base_url = 'https://www.psychologytoday.com/intl/blog-posts?page='

In [135]:
cwd = os.getcwd()
directory = cwd + '\psyco_htmls\\'
try:
    os.mkdir(directory)
except Exception as e:
    print(e)
    
#Write some texts to html in htmls folder

def write_to_html(directory, text:str, order):
    file_name = f"{directory}/page{order}.html"
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(text)

[WinError 183] Cannot create a file when that file already exists: 'd:\\Henson\\Iseng Development\\Web Scraper\\venv\\web_scraper\\psyco_htmls\\'


In [133]:
#Scrap website and then save the result to seperate html files for each page
pages = 10 #pages to scrap
for i in range(pages):
    url = base_url + str(i) #URL plus query to get different pages
    cap_data = requests.get(url, headers=headers)
    write_to_html(directory, cap_data.text, i+1)

In [136]:
# Get all html files in folder htmls into a list
scrap_files = os.listdir(directory)

In [137]:
# Run beautifulsoup for html files and return the object
def beautify(file):
    with open(file, 'r', encoding='utf-8') as f:
        html_text = f.read()
    # print(html_text)
    return bs(html_text, 'html.parser') # return soup

# Extract data from soup and return the result as dictionaries
def extract_info_from_soup(soup: bs):
    results = []
    containers = soup.find_all('div', class_='blog_entry__text')
    for content in containers:
        results.append({
            'title': content.h2.a.text,
            'genre': content.h6.text,
            'publisher': content.p.a.text,
            'publishdate': content.p.text,
            'magazine': content.p.find_all('a')[1].text,
            'url': 'psychologytoday.com' + content.h2.a['href']
        })
    return results

In [138]:
# List of all dictionaries containing the extracted data for each entry
list_result = []

# for each html file, get the data
for scrap in scrap_files:
    soup = beautify(directory + scrap)
    list_result += extract_info_from_soup(soup)
    

In [139]:
uncleaned_data = pd.DataFrame(list_result)
uncleaned_data.dtypes


title          object
genre          object
publisher      object
publishdate    object
magazine       object
url            object
dtype: object

In [140]:
uncleaned_data['publishdate'] = uncleaned_data['publishdate'].str.extract('([A-Za-z]\w+ [0-9]\d{0,1}, [0-9]\d{3})').replace(',','')
uncleaned_data['publishdate'] = uncleaned_data['publishdate'].str.replace(',','')
uncleaned_data['publishdate'] = pd.to_datetime(uncleaned_data['publishdate'])

In [116]:
# uncleaned_data.to_excel('psyco.xlsx')
uncleaned_data.to_csv('psyco.csv')